In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

from cibblbibbl import bbcode

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
w = cibblbibbl.phpwikicode

In [4]:
fname = "c:\\Users\\Szieberth\\Documents\\Repos\\cibblbibbl-data\\cibbl\\page\\achievements.20100926.wiki.txt"

In [5]:
G.init()

In [30]:
with pathlib.Path(fname).open("r", encoding="utf8") as f:
    target_text = f.read()

In [32]:
print(target_text)

{center
[image|https://fumbbl.com/i/596322]%%%
}
%%%


!!!Team Achievements
%%%



<<<

[https://fumbbl.com/i/597778]%%%
__!CabalVision Gold Partner__%%%
''Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer.''

|||

[https://fumbbl.com/i/597779]%%%
__!CabalVision Silver Partner__%%%
''Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer.''

>>>
<<<

[https://fumbbl.com/i/597714]%%%
__Obsidian Chalice__%%%
''Awarded to the most recent winner of the Obsidian Chalice tournament.''

|||

[https://fumbbl.com/i/597715]%%%
__Pearl Chalice__%%%
''Awarded to the most recent winner of the

In [ ]:
teamac = {}
for k, cls in cibblbibbl.achievement.Achievement.registry.items():
    C = cls.defaultconfig_of_group(G)._data
    if not cls.__members__:
        continue
    if not C.get("description"):
        continue
    if cls.subject_typename == "Team":
        teamac[C["name"]] = cls
teamac["CabalVision Gold Partner"] = [
    "https://fumbbl.com/i/597778",
    "!CabalVision Gold Partner",
    "Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer."
]
teamac["CabalVision Silver Partner"] = [
    "https://fumbbl.com/i/597779",
    "!CabalVision Silver Partner",
    "Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer."
]
teamac

In [ ]:
def getcontent(a_cls):
    if isinstance(a_cls, list):
        logo_url, name, description = a_cls
    else:
        C = a_cls.defaultconfig_of_group(G)._data
        logo_url = C.get("logo_url", "")
        logo_url = f'https://fumbbl.com{logo_url}'
        name = C.get("phpwikiname", C["name"])
        description = C.get("phpwikidescription", C["description"])
    return (
        w.url(logo_url) + w.br() + "\n"
        + w.b(name) + w.br() + "\n"
        + w.i(description)
    )

In [66]:
parts = []
gen = iter(sorted(teamac))
for k in gen:
    content1 = getcontent(teamac[k])
    try:
        content2 = getcontent(teamac[next(gen)])
    except StopIteration:
        content2 = ""
    parts.append(w.dualcolumn(content1, content2))
teamaccontent = "\n".join(parts)
print(teamaccontent)

<<<

[https://fumbbl.com/i/597778]%%%
__!CabalVision Gold Partner__%%%
''Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer.''

|||

[https://fumbbl.com/i/597779]%%%
__!CabalVision Silver Partner__%%%
''Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer.''

>>>
<<<

[https://fumbbl.com/i/511916]%%%
__Crushing Victory__%%%
''Awarded whenever a team scores at least 4 times, while the opponent scores no touchdowns at all. Worth 10 Prestige Points to the team, but will 'decay' like normally earned Prestige Points.''

|||

[https://fumbbl.com/i/597714]%%%
__Obsidian Chalice__%%%
''

In [67]:
playerac = collections.defaultdict(set)
for k, cls in cibblbibbl.achievement.Achievement.registry.items():
    if not cls.__members__:
        continue
    D = cls.defaultconfig_of_group(G)
    C = D._data
    bracket = C.get("bracket")
    if not bracket:
        continue
    playerac[bracket].add(cls)

In [68]:
bracketorder = [
    "Development",
    "Performance",
    "Best Players of Winter Tournaments",
    "Best Players of Spring Tournaments",
    "Best Players of Summer Tournaments",
    "Best Players of Autumn Tournaments",
    "Other Best Players",
]

In [69]:
parts = []
for bracket in bracketorder:
    parts.append(f'{w.br()}\n!!{bracket}')
    bracketparts = []
    clss0 = playerac[bracket]
    clss1 = sorted(clss0, key=lambda c: c.defaultconfig_of_group(G)._data.get("bracketsortkey", c.defaultconfig_of_group(G)._data["name"])) 
    gen = iter(clss1)
    for cls in gen:
        content1 = getcontent(cls)
        try:
            content2 = getcontent(next(gen))
        except StopIteration:
            content2 = ""
        bracketparts.append(w.dualcolumn(content1, content2))
    bracketcontent = "\n".join(bracketparts)
    parts.append(bracketcontent)
playeraccontent = "\n\n\n\n\n".join(parts)
print(playeraccontent)

%%%
!!Development




<<<

[https://fumbbl.com/i/531584]%%%
__Legend__%%%
''Awarded to any player reaching 176+ SPP. Player is recognized as a legend of the league. Worth 10 Prestige Points to the team as long as that player is on the team.''

|||

[https://fumbbl.com/i/506476]%%%
__Mega Star Player__%%%
''Awarded to any player reaching 126+ SPP. Player is recognized as a mega star of the league. Worth 10 Prestige Points to the team as long as that player is on the team.''

>>>
<<<

[https://fumbbl.com/i/506474]%%%
__Star Player__%%%
''Awarded to any player reaching 51+ SPP. Player is recognized as one of the absolute stars of the league. Worth 10 Prestige Points to the team as long as that player is on the team.''

|||

[https://fumbbl.com/i/506475]%%%
__Super Star Player__%%%
''Awarded to any player reaching 76+ SPP. Player is recognized as a super star of the league. Worth 10 Prestige Points to the team as long as that player is on the team.''

>>>




%%%
!!Performance




<<<

[ht

In [75]:
parts = [
    w.center(w.url("https://fumbbl.com/i/596322")),
    "\n\n",
    w.br(),
    f'!!!Team Achievements',
    w.br(),
    "\n\n\n",
    teamaccontent,
    "\n\n",
    w.br(),
    f'!!!Player Achievements',
    "\n\n\n",
    playeraccontent,
]
content = "\n".join(parts)
print(content)

{center
[https://fumbbl.com/i/596322]
}



%%%
!!!Team Achievements
%%%




<<<

[https://fumbbl.com/i/597778]%%%
__!CabalVision Gold Partner__%%%
''Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer.''

|||

[https://fumbbl.com/i/597779]%%%
__!CabalVision Silver Partner__%%%
''Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer.''

>>>
<<<

[https://fumbbl.com/i/511916]%%%
__Crushing Victory__%%%
''Awarded whenever a team scores at least 4 times, while the opponent scores no touchdowns at all. Worth 10 Prestige Points to the team, but will 'decay' like normally earned Prestige

In [150]:
"wdadq"[:-1]

'wdad'

In [151]:
Se = max(G.seasons).prev
Se

Season('cibbl', 6, 4)

In [1]:
for name, cls in teamac.items():
    if isinstance(cls, list):
        continue
    else:
        aname = cls.__name__.split("_")[1]
        page = f'CIBBL-Achievement-{aname}'
        m0 = cls.__members__.values()
        if hasattr(cls, "match"):
            m1 = sorted(m0, key=lambda c: -c.match.Id)
        else:
            m1 = sorted(m0, key=lambda c: (-c.season.year.nr, -c.season.nr, c.subject))
        if name == "CrushingVictory":
            header = ["Tournament", "M", "Team", "Roster", "Coach", f'Val']
            header = [w.b(s) for s in header]
            rows = []
            for c in m1:
                date = f'Y{c.season.year.nr} {c.season.name}'
                tournament = w.tournament(c.tournament, f'{date} – ' + re.split(r"[-–]", c.tournament.longname)[-1].strip())
                match = w.url(f'https://fumbbl.com/p/match?id={c.match.Id}', "M")
                team = w.url(f'https://fumbbl.com/p/team?team_id={c.subject.Id}', c.subject.name)
                roster = c.subject.roster_name
                coach = w.url(f'https://fumbbl.com/~{c.subject.coach_name}', c.subject.coach_name)
                prestige = c.prestige(season=Se)
                rows.append([tournament, match, team, roster, coach, prestige])
            stable = table(rows, header=header)
            parts = [
                "{center\n[https://fumbbl.com/i/646307]%%%\n}",
                getcontent(c),
                stable,
                ""
            ]
            s = "\n".join(parts)
            print(s)
        break

NameError: name 'teamac' is not defined